# Criação de banco com médias por município

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

cat_list = []
val_list = []

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Função de classificação de variável**

"Descobre" se a variável é categórica ou numérica

In [4]:
def get_vartypes(df):
    global cat_list, val_list
    cat_list = []
    val_list = []
    for col in df.columns:
        if (df[col].nunique() < 21) & (col not in ['qtde_pessoas', 'qtd_comodos_domic_fam', 'qtd_comodos_dormitorio_fam']):
            cat_list.append(col)
        else:
            val_list.append(col)
            
    return True

**Processamento da base amostral de pessoa**

In [7]:
df_p = pd.DataFrame()
familias = []
varset = False

for chunk in pd.read_csv("/content/drive/MyDrive/The Sound of Drums/Notebooks/data/base_amostra_pessoa_201812.csv", sep=';', decimal=',', chunksize=500000):
    c = chunk.drop(columns=['estrato', 'classf', 'id_pessoa', 'cod_parentesco_rf_pessoa', 'peso.fam', 'peso.pes'])
    c = c[(c.idade > 13) & (c.idade < 41) & (c.cod_sexo_pessoa == 2)]
    familias = familias + c.id_familia.unique().tolist()
    
    if varset == False: varset = get_vartypes(c)
        
    c = pd.get_dummies(c, columns=cat_list)
    
    numero = c.groupby('cd_ibge', as_index=False)[['id_familia']].count().rename(columns={'id_familia':'qnt'})
    c = c.groupby('cd_ibge', as_index=False).mean()
    meancols = list(c.columns)
    c = c.merge(numero, on='cd_ibge')
    c[meancols[2:]] = c[meancols[2:]].multiply(c['qnt'], axis='index')
    df_p = df_p.append(c).fillna(0)
    print(df_p.shape)

(5028, 116)
(10058, 116)
(15112, 116)
(20157, 116)
(25205, 116)
(30213, 116)
(35256, 116)
(40255, 116)
(45268, 116)
(50315, 116)
(55371, 116)
(60397, 116)
(65433, 116)
(70463, 116)
(75483, 116)
(80530, 116)
(85550, 116)
(90579, 116)
(95612, 116)
(100645, 116)
(105657, 116)
(110694, 116)
(115736, 116)
(120783, 116)
(125808, 116)
(130576, 116)


In [8]:
df_pp = df_p.groupby('cd_ibge').sum()
df_pp = df_pp.divide(df_pp.qnt, axis=0).drop(columns=['id_familia', 'qnt'])

**Processamento de base amostral de família**

In [10]:
# Base de amostras familiares anonimizadas do CADU 
df = pd.DataFrame()
varset = False
for chunk in pd.read_csv("/content/drive/MyDrive/The Sound of Drums/Notebooks/data/base_amostra_familia_201812.csv", sep=';', decimal=',', chunksize=500000, low_memory=False):
    c = chunk.drop(columns=['estrato', 'classf', 'dat_cadastramento_fam', 'dat_alteracao_fam', 'dat_atualizacao_familia', 
                            'peso.fam', 'cod_centro_assist_fam', 'cod_eas_fam'])
    c = c[(c.id_familia.isin(familias))]
    
    if varset == False: varset = get_vartypes(c)
        
    c = pd.get_dummies(c, columns=cat_list)
    numero = c.groupby('cd_ibge', as_index=False)[['id_familia']].count().rename(columns={'id_familia':'qnt'})
    c = c.groupby('cd_ibge', as_index=False).mean()
    meancols = list(c.columns)
    c = c.merge(numero, on='cd_ibge')
    c[meancols[2:]] = c[meancols[2:]].multiply(c['qnt'], axis='index')
    
    df = df.append(c).fillna(0)
    print(df.shape)

(2171, 75)
(4500, 75)
(6373, 75)
(6786, 75)
(7055, 75)
(7223, 75)
(7317, 75)
(7360, 77)
(7375, 77)
(7377, 77)


In [11]:
df_pes = df.groupby('cd_ibge').sum()
df_pes = df_pes.divide(df_pes.qnt, axis=0).drop(columns=['id_familia', 'qnt'])

In [12]:
df_final = df_pp.join(df_pes).reset_index()

In [13]:
# Base de taxa de nascimentos prematuos por município (post-SINASC)
ptb_rate_df = pd.read_csv("/content/drive/MyDrive/The Sound of Drums/Notebooks/data/ptb_by_municipality.csv")
df_final = df_final.merge(ptb_rate_df, on='cd_ibge')

In [14]:
df_final

,cd_ibge,idade,val_remuner_emprego_memb,val_renda_bruta_12_meses_memb,val_renda_doacao_memb,val_renda_aposent_memb,val_renda_seguro_desemp_memb,val_renda_pensao_alimen_memb,val_outras_rendas_memb,cod_sexo_pessoa_2,cod_raca_cor_pessoa_1.0,cod_raca_cor_pessoa_2.0,cod_raca_cor_pessoa_3.0,cod_raca_cor_pessoa_4.0,cod_raca_cor_pessoa_5.0,cod_local_nascimento_pessoa_1.0,cod_local_nascimento_pessoa_2.0,cod_local_nascimento_pessoa_3.0,cod_certidao_registrada_pessoa_1,cod_certidao_registrada_pessoa_2,cod_certidao_registrada_pessoa_3,cod_certidao_registrada_pessoa_4,cod_deficiencia_memb_1,cod_deficiencia_memb_2,cod_sabe_ler_escrever_memb_1.0,cod_sabe_ler_escrever_memb_2.0,ind_frequenta_escola_memb_1.0,ind_frequenta_escola_memb_2.0,ind_frequenta_escola_memb_3.0,ind_frequenta_escola_memb_4.0,cod_escola_local_memb_1.0,cod_escola_local_memb_2.0,cod_curso_frequenta_memb_1.0,cod_curso_frequenta_memb_2.0,cod_curso_frequenta_memb_3.0,cod_curso_frequenta_memb_4.0,cod_curso_frequenta_memb_5.0,cod_curso_frequenta_memb_6.0,cod_curso_frequenta_memb_7.0,cod_curso_frequenta_memb_8.0,cod_curso_frequenta_memb_9.0,cod_curso_frequenta_memb_10.0,cod_curso_frequenta_memb_11.0,cod_curso_frequenta_memb_12.0,cod_curso_frequenta_memb_13.0,cod_curso_frequenta_memb_14.0,cod_ano_serie_frequenta_memb_1.0,cod_ano_serie_frequenta_memb_2.0,cod_ano_serie_frequenta_memb_3.0,cod_ano_serie_frequenta_memb_4.0,cod_ano_serie_frequenta_memb_5.0,cod_ano_serie_frequenta_memb_6.0,cod_ano_serie_frequenta_memb_7.0,cod_ano_serie_frequenta_memb_8.0,cod_ano_serie_frequenta_memb_9.0,cod_ano_serie_frequenta_memb_10.0,cod_curso_frequentou_pessoa_memb_1.0,cod_curso_frequentou_pessoa_memb_2.0,cod_curso_frequentou_pessoa_memb_3.0,cod_curso_frequentou_pessoa_memb_4.0,cod_curso_frequentou_pessoa_memb_5.0,cod_curso_frequentou_pessoa_memb_6.0,cod_curso_frequentou_pessoa_memb_7.0,cod_curso_frequentou_pessoa_memb_8.0,cod_curso_frequentou_pessoa_memb_9.0,cod_curso_frequentou_pessoa_memb_10.0,cod_curso_frequentou_pessoa_memb_11.0,cod_curso_frequentou_pessoa_memb_12.0,cod_curso_frequentou_pessoa_memb_13.0,cod_curso_frequentou_pessoa_memb_14.0,cod_curso_frequentou_pessoa_memb_15.0,cod_ano_serie_frequentou_memb_1.0,cod_ano_serie_frequentou_memb_2.0,cod_ano_serie_frequentou_memb_3.0,cod_ano_serie_frequentou_memb_4.0,cod_ano_serie_frequentou_memb_5.0,cod_ano_serie_frequentou_memb_6.0,cod_ano_serie_frequentou_memb_7.0,cod_ano_serie_frequentou_memb_8.0,cod_ano_serie_frequentou_memb_9.0,cod_ano_serie_frequentou_memb_10.0,cod_concluiu_frequentou_memb_1.0,cod_concluiu_frequentou_memb_2.0,cod_trabalhou_memb_1.0,cod_trabalhou_memb_2.0,cod_afastado_trab_memb_1.0,cod_afastado_trab_memb_2.0,cod_agricultura_trab_memb_1.0,cod_agricultura_trab_memb_2.0,cod_principal_trab_memb_1.0,cod_principal_trab_memb_2.0,cod_principal_trab_memb_3.0,cod_principal_trab_memb_4.0,cod_principal_trab_memb_5.0,cod_principal_trab_memb_6.0,cod_principal_trab_memb_7.0,cod_principal_trab_memb_8.0,cod_principal_trab_memb_9.0,cod_principal_trab_memb_10.0,cod_principal_trab_memb_11.0,cod_trabalho_12_meses_memb_1.0,cod_trabalho_12_meses_memb_2.0,qtd_meses_12_meses_memb_1.0,qtd_meses_12_meses_memb_2.0,qtd_meses_12_meses_memb_3.0,qtd_meses_12_meses_memb_4.0,qtd_meses_12_meses_memb_5.0,qtd_meses_12_meses_memb_6.0,qtd_meses_12_meses_memb_7.0,qtd_meses_12_meses_memb_8.0,qtd_meses_12_meses_memb_9.0,qtd_meses_12_meses_memb_10.0,qtd_meses_12_meses_memb_11.0,qtd_meses_12_meses_memb_12.0,vlr_renda_media_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,qtde_pessoas,cod_local_domic_fam_1.0,cod_local_domic_fam_2.0,cod_especie_domic_fam_1.0,cod_especie_domic_fam_2.0,cod_especie_domic_fam_3.0,cod_material_piso_fam_1.0,cod_material_piso_fam_2.0,cod_material_piso_fam_3.0,cod_material_piso_fam_4.0,cod_material_piso_fam_5.0,cod_material_piso_fam_6.0,cod_material_piso_fam_7.0,cod_material_domic_fam_1.0,cod_material_domic_fam_2.0,cod_material_domic_fam_3.0,cod_material_domic_fam_4.0,cod_material_domic_fam_5.0,cod_material_domic_fam_6.0,cod_material_domic_fam_7.

In [15]:
df_final.to_csv("/content/drive/MyDrive/The Sound of Drums/Notebooks/data/pos_means1.csv", index=False)